# Oppgave 1
## Hvor mange oppføringer er det totalt?

In [2]:
import pandas as pd
filename = "O1-UserAgents.csv"

In [3]:
df = pd.read_csv(filename, delimiter=",")
print("Datasettet inneholder {nrows} oppføringer".format(nrows = len(df.index)))

Datasettet inneholder 5345 oppføringer


# Oppgave 2
## Datasettet inneholder syntaksfeil, hvilke linjer?

Jeg er litt usikker på hva som menes med syntaksfeil i denne oppgaven. <br>
For det første, lese inn hver linje og splitte datafeltene med komma som separator fungerer dårlig, siden feltet "http.http_user_agent" er en string som kan inneholde komma.
<br>
Dette er ikke noe problem; `pandas.read_csv()` ser ut til å jobbe rundt dette problemet ganske enkelt.
<br><br>
Jeg regulære uttrykk for å sjekke om alle radene er i formatet "string","string","string","string" <br>
Jeg lar vær å sjekke om siste feltet er omgitt i "", da det ser ut som alle oppføringene har det siste feltet uten "".

In [4]:
import re

data_format = re.compile(r'^"([^"]*)","([^"]*)","([^"]*)",([^"]+)$')

with open(filename, 'r') as file:
    lines = file.readlines()
error_indices= []
error_lines = []

for i, line in enumerate(lines[1:], start=1):
    match = data_format.match(line)
    if not match:
        error_indices.append(i)
        error_lines.append(line)

print("Det er totalt {n_errors} linjer som har syntaksfeil".format(n_errors = len(error_indices)))
print("Indeksene til disse linjene er {inds}".format(inds = error_indices))
print("Nedenfor kan dere lese linjene med syntaksfeil:")
for line in zip(error_indices, error_lines):
    print(line)

Det er totalt 2 linjer som har syntaksfeil
Indeksene til disse linjene er [756, 4333]
Nedenfor kan dere lese linjene med syntaksfeil:
(756, '"2021-03-04T17:35:31.405+0100",Mozilla/4.0 (compatible; ms-office; MSOffice 16)","HTTP/1.1",GET\n')
(4333, '"2021-03-04T17:35:02.789+0100Mozilla/4.0 (compatible; ms-office; MSOffice 16)","HTTP/1.1",GET\n')


Oppføring 756 mangler " i starten av feltet http.http_user_agent <br>
Oppføring 4333 mangler komma som separerer "_time" og "http.http_user_agent" <br>

Retter feilen i innlest dataframe manuelt. Denne metoden er selvsagt ikke skalerbart.

In [5]:
df.loc[755, "http.http_user_agent"]= "Mozilla/4.0 (compatible; ms-office; MSOffice 16)"
df.loc[4332]= ['2021-03-04T17:35:02.789+0100','Mozilla/4.0 (compatible; ms-office; MSOffice 16)','HTTP/1.1','GET']

# Oppgave 3
Hvor mange unike User-Agents eksisterer i datasettet?

In [6]:
n_unq_useragents = df['http.http_user_agent'].nunique()
print("Det er totalt {n_unq_useragents} unike user-agents i datasettet".format(n_unq_useragents = n_unq_useragents))


Det er totalt 31 unike user-agents i datasettet


# Oppgave 4
## Hvor mange unike User-Agents har en forekomst som er større enn den gjennomsnittlige forekomsten?

In [7]:
import numpy as np
agent_counts = df['http.http_user_agent'].value_counts()
avg_count = np.average(agent_counts.values)
agents_above_avg = agent_counts.loc[agent_counts.values > avg_count]


print("Gjennomsnittlig forekomst av unik user-agent er {avg}".format(avg = avg_count))
print("Følgende user_agents har forekomst større enn gjennomsnittlig:")
#for agent_name in agents_above_avg.index: print(agent_name)
agents_above_avg

Gjennomsnittlig forekomst av unik user-agent er 172.41935483870967
Følgende user_agents har forekomst større enn gjennomsnittlig:


http.http_user_agent
Mozilla/4.0 (compatible; ms-office; MSOffice 16)                                                                                             2173
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36                           799
Mozilla/5.0 (X11; Linux armv7l) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36 CrKey/1.49.230269                    581
Mozilla/5.0 (iPhone; CPU iPhone OS 14_4 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Mobile/15E148 Safari/604.1     237
Mozilla/5.0 (X11; Linux aarch64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 CrKey/1.50.230474                  232
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:48.0) Gecko/20100101 Firefox/48.0                                                                      230
Name: count, dtype: int64